In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score
import numpy as np

## 1. Figure out your question

The question we want to answer using machine learning is: Can naming trends accurately predict religiosity in a region?

## 2. Obtain a labeled dataset

The name data we are using comes from the Social Security Administration Database. 
The religous data is compiled from various sources that include: Religious Landscape Survey, American National Election Studies, Churches and Church Membership in the United States, History of American Religion, and the United States Census of American Religion. 
The Bible names were scraped from Wikipedia. 

In [5]:
rel = pd.read_csv("test.csv")
print(rel.head())

  state  year  percentbible  percentAaliyah  percentAaron  percentAbbey  \
0    AK  1971      0.201347        0.000000      0.006558             0   
1    AK  2008      0.235185        0.001684      0.004377             0   
2    AL  1916      0.168510        0.000000      0.000000             0   
3    AL  1926      0.175586        0.000000      0.000000             0   
4    AL  1952      0.168864        0.000000      0.000000             0   

   percentAbbie  percentAbbigail  percentAbby  percentAbdirahman  ...  \
0             0              0.0          0.0                  0  ...   
1             0              0.0          0.0                  0  ...   
2             0              0.0          0.0                  0  ...   
3             0              0.0          0.0                  0  ...   
4             0              0.0          0.0                  0  ...   

   percentZia  percentZion  percentZoe  percentZoey  percentZola  percentZora  \
0           0          0.0   

In [7]:
rel.dropna(subset=['christian'], inplace=True)
rel.drop(columns = ["Validation"])
rel = pd.get_dummies(rel, columns=['state', 'year'])
rel.describe()

,percentbible,percentAaliyah,percentAaron,percentAbbey,percentAbbie,percentAbbigail,percentAbby,percentAbdirahman,percentAbel,percentAbigail,...,state_WI,state_WV,state_WY,year_1916,year_1926,year_1952,year_1971,year_2007,year_2008,year_2016
count,283.000000,283.000000,283.000000,283.0,283.0,283.000000,283.000000,283.0,283.000000,283.000000,...,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000
mean,0.171623,0.000387,0.001294,0.0,0.0,0.000004,0.000038,0.0,0.000099,0.001517,...,0.021201,0.017668,0.017668,0.169611,0.169611,0.173145,0.176678,0.173145,0.007067,0.130742
std,0.027602,0.000693,0.001579,0.0,0.0,0.000063,0.000232,0.0,0.000359,0.002449,...,0.144310,0.131974,0.131974,0.375956,0.375956,0.379043,0.382072,0.379043,0.083917,0.337715
min,0.105697,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.153710,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.172249,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.187223,0.001031,0.002384,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.003626,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.257951,0.003000,0.006857,0.0,0.0,0.001062,0.002288,0.0,0.001888,0.011745,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
y = rel['christian']
X = rel[[x for x in rel.columns if x !='christian']]

## 3. Divide into training and set sets

In [9]:
xTrain, xTest, yTrain, yTest = train_test_split(X, y,test_size=0.2,random_state=42)

## 4. Pick an appropriate method
We will begin with Lasso.

In [17]:
lasso = Lasso(alpha=0.001, max_iter=100000).fit(xTrain, yTrain)

#score

yhat = lasso.predict(xTest)
print("R2 Score:", r2_score(yTest, yhat))
print()
print("Predicted:")
print(yhat[0:5])
print("Actual:")
print(yTest[0:5])
print()
print("Number of features used:", np.sum(lasso.coef_ != 0))
print()
scores = cross_val_score(lasso,xTrain,yTrain,cv=5)
print("Cross-validation scores: {}".format(scores))
print("Average cross-validation score: {:.2f}".format(scores.mean()))

R2 Score: 0.615636812077325

Predicted:
[0.59052542 0.76124923 0.56455134 0.37316849 0.79589368]
Actual:
9      0.531862
246    0.758335
139    0.510789
207    0.272718
75     0.794677
Name: christian, dtype: float64

Number of features used: 29

Cross-validation scores: [0.56284709 0.54559543 0.57579911 0.62693311 0.73235929]
Average cross-validation score: 0.61


## 5. Choose regularization parameters via cross-validation on the training set

In [18]:
alpha_grid = {'alpha': [.0001, .001, .002, .004, .006, .008, .01, .012, .014, .016 ,.018, .02 ],'max_iter': [100000]}
grid_search = GridSearchCV(Lasso(),alpha_grid,cv=5,scoring=make_scorer(r2_score, greater_is_better=True))
best_model=grid_search.fit(xTrain,yTrain)
print("Best alpha: ",best_model.best_estimator_.get_params()['alpha'])

C:\Users\lisa\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best alpha:  0.0001


In [22]:
BetterLasso = Lasso(alpha=0.0001, max_iter=100000).fit(xTrain, yTrain)
yhat = BetterLasso.predict(xTest)
print("R2 Score:", r2_score(yTest, yhat))
print()
print("Predicted:")
print(yhat[0:5])
print("Actual:")
print(yTest[0:5])
print()
print("Number of features used:", np.sum(lasso.coef_ != 0))
print()
scores = cross_val_score(lasso,xTrain,yTrain,cv=5)
print("Cross-validation scores: {}".format(scores))
print("Average cross-validation score: {:.2f}".format(scores.mean()))

R2 Score: 0.6853350607365616

Predicted:
[0.58952429 0.83747531 0.5722812  0.2960483  0.77782597]
Actual:
9      0.531862
246    0.758335
139    0.510789
207    0.272718
75     0.794677
Name: christian, dtype: float64

Number of features used: 29

Cross-validation scores: [0.56284709 0.54559543 0.57579911 0.62693311 0.73235929]
Average cross-validation score: 0.61


## 6. Fit model on whole training set using the cross-validated parameters

## 7. Evaluate model by applying it to test set

## 8. Repeat 4-7 for other methods

In [20]:
#Ridge regression
alpha_grid = {'alpha': [.0001, .001, .002, .004, .006, .008, .01, .012, .014, .016 ,.018, .02 ],'max_iter': [100000]}
grid_search = GridSearchCV(Ridge(),alpha_grid,cv=5,scoring=make_scorer(r2_score, greater_is_better=True))
best_model=grid_search.fit(xTrain,yTrain)
print("Best alpha: ",best_model.best_estimator_.get_params()['alpha'])

Best alpha:  0.0001


C:\Users\lisa\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [24]:
ridge = Ridge(alpha=.0001).fit(xTrain, yTrain)
yhat = ridge.predict(xTest)
print("R2 Score:", r2_score(yTest, yhat))
print()
print("Predicted:")
print(yhat[0:5])
print("Actual:")
print(yTest[0:5])
print()
print("Number of features used:", np.sum(ridge.coef_ != 0))
print()
scores = cross_val_score(ridge,xTrain,yTrain,cv=5)
print("Cross-validation scores: {}".format(scores))
print("Average cross-validation score: {:.2f}".format(scores.mean()))

R2 Score: 0.8086870241388175

Predicted:
[0.54388578 0.78443338 0.47645359 0.33680923 0.79781547]
Actual:
9      0.531862
246    0.758335
139    0.510789
207    0.272718
75     0.794677
Name: christian, dtype: float64

Number of features used: 1849

Cross-validation scores: [0.68212783 0.82770746 0.84607282 0.86556941 0.8978952 ]
Average cross-validation score: 0.82


In [26]:
#Lasso and Ridge excluding Bible variable
rel2 = rel.drop(columns=['percentbible'])
rel2.head()

,percentAaliyah,percentAaron,percentAbbey,percentAbbie,percentAbbigail,percentAbby,percentAbdirahman,percentAbel,percentAbigail,percentAbigale,...,state_WI,state_WV,state_WY,year_1916,year_1926,year_1952,year_1971,year_2007,year_2008,year_2016
0,0.000000,0.006558,0,0,0.0,0.0,0,0.0,0.000000,0,...,0,0,0,0,0,0,1,0,0,0
1,0.001684,0.004377,0,0,0.0,0.0,0,0.0,0.006734,0,...,0,0,0,0,0,0,0,0,1,0
2,0.000000,0.000000,0,0,0.0,0.0,0,0.0,0.000000,0,...,0,0,0,1,0,0,0,0,0,0
3,0.000000,0.000000,0,0,0.0,0.0,0,0.0,0.000000,0,...,0,0,0,0,1,0,0,0,0,0
4,0.000000,0.000000,0,0,0.0,0.0,0,0.0,0.000000,0,...,0,0,0,0,0,1,0,0,0,0


In [27]:
y = rel2['christian']
X = rel2[[x for x in rel2.columns if x !='christian']]
x_Train, x_Test, y_Train, y_Test = train_test_split(X, y,test_size=0.2,random_state=1)

In [28]:
alpha_grid = {'alpha': [.0001, .001, .002, .004, .006, .008, .01, .012, .014, .016 ,.018, .02 ],'max_iter': [100000]}
grid_search = GridSearchCV(Lasso(),alpha_grid,cv=5,scoring=make_scorer(r2_score, greater_is_better=True))
best_model=grid_search.fit(x_Train,y_Train)
print("Best alpha: ",best_model.best_estimator_.get_params()['alpha'])

Best alpha:  0.001


C:\Users\lisa\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [29]:
BetterLasso = Lasso(alpha=0.001, max_iter=100000).fit(x_Train, y_Train)
yhat = BetterLasso.predict(x_Test)
print("R2 Score:", r2_score(y_Test, yhat))
print()
print("Predicted:")
print(yhat[0:5])
print("Actual:")
print(y_Test[0:5])
print()
print("Number of features used:", np.sum(BetterLasso.coef_ != 0))
print()
scores = cross_val_score(BetterLasso,x_Train,y_Train,cv=5)
print("Cross-validation scores: {}".format(scores))
print("Average cross-validation score: {:.2f}".format(scores.mean()))

R2 Score: 0.5317987801856817

Predicted:
[0.8130718  0.79379215 0.52382987 0.59346289 0.53625846]
Actual:
99     0.899727
260    0.656362
62     0.624104
102    0.688436
259    0.520894
Name: christian, dtype: float64

Number of features used: 29

Cross-validation scores: [0.75168341 0.5489692  0.75052847 0.72503825 0.74540702]
Average cross-validation score: 0.70


In [30]:
#Ridge regression
alpha_grid = {'alpha': [.0001, .001, .002, .004, .006, .008, .01, .012, .014, .016 ,.018, .02 ],'max_iter': [100000]}
grid_search = GridSearchCV(Ridge(),alpha_grid,cv=5,scoring=make_scorer(r2_score, greater_is_better=True))
best_model=grid_search.fit(x_Train,y_Train)
print("Best alpha: ",best_model.best_estimator_.get_params()['alpha'])

Best alpha:  0.0001


In [31]:
ridge = Ridge(alpha=.0001).fit(x_Train, y_Train)
yhat = ridge.predict(x_Test)
print("R2 Score:", r2_score(y_Test, yhat))
print()
print("Predicted:")
print(yhat[0:5])
print("Actual:")
print(y_Test[0:5])
print()
print("Number of features used:", np.sum(ridge.coef_ != 0))
print()
scores = cross_val_score(ridge,x_Train,y_Train,cv=5)
print("Cross-validation scores: {}".format(scores))
print("Average cross-validation score: {:.2f}".format(scores.mean()))

R2 Score: 0.8099493098666078

Predicted:
[0.87314167 0.73069416 0.56499577 0.6864763  0.56533665]
Actual:
99     0.899727
260    0.656362
62     0.624104
102    0.688436
259    0.520894
Name: christian, dtype: float64

Number of features used: 1715

Cross-validation scores: [0.75892982 0.86465118 0.77839687 0.78871531 0.82228844]
Average cross-validation score: 0.80


## 9. Apply the chosen method to new observations for which we have no labels

In [29]:
out = pd.read_csv("ApplyData.csv")

In [31]:
lasso.predict(out)

TypeError: 'DataFrame' object is not callable